# Overview

Here, we will demonstrate how to carry out geometry optimizations using ASE. For this, we need to pick an energy model --- which in ASE is called a calculator. The calculator is the simulation engine that, given an atomic configuration, will return the energy of the system (as well as forces and, for solids, the unit cell stresses).

For this exercise, we will mainly use a "machine learning potential". We will cover machine learning potentials in greater detail later in the course, but it is a machine learning model trained to reproduce density functional theory energy, forces, and stresses at a low computational cost. The model we will use in this exercise is called TensorNet-MatPES-r2SCAN from https://arxiv.org/abs/2503.04070.


# Setup


There is a bug with the ASE GUI in ase 3.27.0, so until 3.27.1 is released, we will install the development version.


In [22]:
%uv pip install ase@git+https://gitlab.com/ase/ase.git

Note: you may need to restart the kernel to use updated packages.


Using Python 3.13.11 environment at: C:\Users\asros\miniconda3\envs\cms
   Updating https://gitlab.com/ase/ase.git (HEAD)
    Updated https://gitlab.com/ase/ase.git (9e022fa5b8d123bc1785145661104a4f893a10c0)
Resolved 13 packages in 11.27s
Audited 13 packages in 5ms


For this demonstration, we also need to install a code called matcalc. This will give us the machine learning potential to use as our energy and force calculator.


In [ ]:
%uv pip install matcalc[matgl]

Note: you may need to restart the kernel to use updated packages.


# Demonstration


First, we will need to define our atomic system. We will study a porous material called a metal-organic framework, specifically MOF-5.


In [2]:
from ase.io import read

atoms = read("mof5.cif")

In [19]:
atoms

Atoms(symbols='C192H96O104Zn32', pbc=True, cell=[[25.880232683010675, 4.250589686636913e-08, 7.847959566961198e-08], [4.250537110201552e-08, 25.880232947121996, 1.6312256899758683e-08], [7.84792244170589e-08, 1.6328391036896475e-08, 25.880232785123376]], spacegroup_kinds=..., calculator=LennardJones(...))

In [3]:
from ase.visualize import view

view(atoms)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

Now we need to define a calculator. ASE comes with many pre-made calculators (https://ase-lib.org/ase/calculators/calculators.html), and there are many external codes that ship their own ASE calculators.


We will load the TensorNet-MatPES-r2SCAN calculator


In [4]:
from matcalc import load_fp

calc = load_fp("TensorNet-MatPES-r2SCAN-v2025.1-PES")

Now we can attach the calculator to the `Atoms` object.


In [5]:
atoms.calc = calc

We now have everything we need to run a calculation. Let's start with running a static (single-point) calculation.


In [6]:
e_initial = atoms.get_potential_energy()
print(e_initial)


-3339.190185546875


It is always important to understand the units of any given code. In the case of ASE, energies are given in eV (https://ase-lib.org/ase/units.html). There are 96.49 kJ/mol per eV.

An energy on its own is meaningless. It must be referenced to something. We will come back to this.


The structure we loaded not be a minimum energy structure. We must run a geometry optimization to obtain this. We can do this in ASE as well.

ASE comes with many optimizers. We will use BFGS.


In [7]:
from ase.optimize import BFGS

# Set up the BFGS optimizer and specify output file
opt = BFGS(atoms, trajectory="opt.traj")

In [8]:
opt.run(fmax=0.01)  # Run until the maximum force is less than 0.01 eV/Å

      Step     Time          Energy          fmax
BFGS:    0 11:39:37    -3339.190186        0.343954
BFGS:    1 11:39:39    -3339.353516        0.215392
BFGS:    2 11:39:41    -3339.451660        0.134192
BFGS:    3 11:39:43    -3339.477539        0.099507
BFGS:    4 11:39:44    -3339.499756        0.062971
BFGS:    5 11:39:46    -3339.511963        0.051177
BFGS:    6 11:39:47    -3339.516113        0.017784
BFGS:    7 11:39:48    -3339.516602        0.010649
BFGS:    8 11:39:50    -3339.516602        0.009292


np.True_

In [9]:
e_final = atoms.get_potential_energy()  # get the optimized energy
print(e_final - e_initial)  # print the energy difference

-0.326416015625


In [10]:
atoms.get_forces()

array([[-0.00248075, -0.00248073,  0.00248153],
       [ 0.00248719,  0.0024947 , -0.00250016],
       [ 0.00250266,  0.00249279,  0.00248121],
       ...,
       [-0.00409735,  0.00585589,  0.00585571],
       [-0.00414147,  0.00580159, -0.00580242],
       [ 0.00411937, -0.00582212,  0.0058229 ]],
      shape=(424, 3), dtype=float32)

It looks like we found a lower energy structure. Let's take a look at the optimization trajectory now.


In [11]:
trajectory = read("opt.traj", index=":")  # index=":" means read all the frames

In [12]:
from ase.visualize import view

view(trajectory)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

If you look closely, you will see that the atomic positions updated and the energy went down. The forces went down too.

But note that the unit cell shape and size did not change. In addition to the atomic positions, the unit cell can also be optimized. After all, if you made the unit cell too big, that may not be the lowest energy structure. We can optimize both the positions and unit cell in ASE simultaneously.

In ASE, this is done by wrapping the `Atoms` object in a "Cell Filter". Otherwise, the procedure remains largely the same.


In [13]:
from ase.filters import FrechetCellFilter

# this tells ASE to optimize both atomic positions and cell parameters
fcf = FrechetCellFilter(atoms)


In [14]:
opt = BFGS(fcf, trajectory="opt2.traj")
opt.run(fmax=0.01)

      Step     Time          Energy          fmax
BFGS:    0 11:39:51    -3339.516602        0.099594
BFGS:    1 11:39:53    -3339.516846        0.099030
BFGS:    2 11:39:55    -3339.522949        0.093084
BFGS:    3 11:39:57    -3339.525879        0.090448
BFGS:    4 11:39:58    -3339.532959        0.086335
BFGS:    5 11:40:00    -3339.540771        0.085053
BFGS:    6 11:40:01    -3339.551270        0.086359
BFGS:    7 11:40:02    -3339.564941        0.089813
BFGS:    8 11:40:04    -3339.586670        0.094768
BFGS:    9 11:40:05    -3339.617432        0.098252
BFGS:   10 11:40:07    -3339.650391        0.102750
BFGS:   11 11:40:08    -3339.684570        0.110435
BFGS:   12 11:40:10    -3339.718994        0.111838
BFGS:   13 11:40:11    -3339.754395        0.109309
BFGS:   14 11:40:12    -3339.788086        0.103902
BFGS:   15 11:40:13    -3339.822510        0.096853
BFGS:   16 11:40:15    -3339.855957        0.088642
BFGS:   17 11:40:16    -3339.888672        0.079743
BFGS:   18 11:

np.True_

In [15]:
atoms.get_potential_energy()  # get the optimized energy

-3340.025146484375

In [16]:
view(read("opt2.traj", index=":"))

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

Finally, we will note that there are many ASE calculators, some based on empirical potentials, some based on ML models, some based on quantum chemistry, and so on. An incomplete list can be found at https://ase-lib.org/ase/calculators/calculators.html. It is easy to swap to a different calculator and to do the same procedure as above.

For instance, we could run a calculation with a Lennard-Jones potential in exactly the same way. Note: Of course, the LJ potential we have chosen here is completely unphysical.


In [17]:
from ase.calculators.lj import LennardJones

In [ ]:
# We will run a toy single-point calculation with an LJ potential
# This is completely unphysical because we have arbitrarily chosen sigma=1.0 and epsilon=1.0
# But it serves to illustrate how to set up a different calculator and also how
# to specify keyword arguments

atoms.calc = LennardJones(sigma=1.0, epsilon=1.0)
atoms.get_potential_energy()

np.float64(-269.3023998243027)